In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.\
    builder.\
    master('local').\
    appName('clustering-basics').\
    getOrCreate()

In [2]:
from pyspark.ml.clustering import KMeans

In [3]:
data = spark.read.format('libsvm').\
    load('D:/learn-ab/learning-PySpark/sample-data/sample-kmeans-data.txt')

In [4]:
data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|           (3,[],[])|
|  1.0|(3,[0,1,2],[0.1,0...|
|  2.0|(3,[0,1,2],[0.2,0...|
|  3.0|(3,[0,1,2],[9.0,9...|
|  4.0|(3,[0,1,2],[9.1,9...|
|  5.0|(3,[0,1,2],[9.2,9...|
+-----+--------------------+



In [5]:
data.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)



In [6]:
final_data = data.select('features')
final_data.show()

+--------------------+
|            features|
+--------------------+
|           (3,[],[])|
|(3,[0,1,2],[0.1,0...|
|(3,[0,1,2],[0.2,0...|
|(3,[0,1,2],[9.0,9...|
|(3,[0,1,2],[9.1,9...|
|(3,[0,1,2],[9.2,9...|
+--------------------+



In [7]:
?KMeans

Init signature:
KMeans(
    *,
    featuresCol: str = 'features',
    predictionCol: str = 'prediction',
    k: int = 2,
    initMode: str = 'k-means||',
    initSteps: int = 2,
    tol: float = 0.0001,
    maxIter: int = 20,
    seed: Optional[int] = None,
    distanceMeasure: str = 'euclidean',
    weightCol: Optional[str] = None,
    solver: str = 'auto',
    maxBlockSizeInMB: float = 0.0,
)
Docstring:     
K-means clustering with a k-means++ like initialization mode
(the k-means|| algorithm by Bahmani et al).

.. versionadded:: 1.5.0

Examples
--------
>>> from pyspark.ml.linalg import Vectors
>>> data = [(Vectors.dense([0.0, 0.0]), 2.0), (Vectors.dense([1.0, 1.0]), 2.0),
...         (Vectors.dense([9.0, 8.0]), 2.0), (Vectors.dense([8.0, 9.0]), 2.0)]
>>> df = spark.createDataFrame(data, ["features", "weighCol"])
>>> kmeans = KMeans(k=2)
>>> kmeans.setSeed(1)
KMeans...
>>> kmeans.setWeightCol("weighCol")
KMeans...
>>> kmeans.setMaxIter(10)
KMeans...
>>> kmeans.getMaxIter()
10
>>> km

In [8]:
kmeans = KMeans().setK(2).setSeed(1)

In [9]:
kmeans_model = kmeans.fit(final_data)

In [10]:
kmeans_pred = kmeans_model.transform(final_data)

In [11]:
kmeans_pred.show()

+--------------------+----------+
|            features|prediction|
+--------------------+----------+
|           (3,[],[])|         1|
|(3,[0,1,2],[0.1,0...|         1|
|(3,[0,1,2],[0.2,0...|         1|
|(3,[0,1,2],[9.0,9...|         0|
|(3,[0,1,2],[9.1,9...|         0|
|(3,[0,1,2],[9.2,9...|         0|
+--------------------+----------+



In [12]:
cluster_centers = kmeans_model.clusterCenters()
cluster_centers

[array([9.1, 9.1, 9.1]), array([0.1, 0.1, 0.1])]

In [13]:
from pyspark.ml.evaluation import ClusteringEvaluator

In [14]:
?ClusteringEvaluator

Init signature:
ClusteringEvaluator(
    *,
    predictionCol: str = 'prediction',
    featuresCol: str = 'features',
    metricName: 'ClusteringEvaluatorMetricType' = 'silhouette',
    distanceMeasure: str = 'squaredEuclidean',
    weightCol: Optional[str] = None,
)
Docstring:     
Evaluator for Clustering results, which expects two input
columns: prediction and features. The metric computes the Silhouette
measure using the squared Euclidean distance.

The Silhouette is a measure for the validation of the consistency
within clusters. It ranges between 1 and -1, where a value close to
1 means that the points in a cluster are close to the other points
in the same cluster and far from the points of the other clusters.

.. versionadded:: 2.3.0

Examples
--------
>>> from pyspark.ml.linalg import Vectors
>>> featureAndPredictions = map(lambda x: (Vectors.dense(x[0]), x[1]),
...     [([0.0, 0.5], 0.0), ([0.5, 0.0], 0.0), ([10.0, 11.0], 1.0),
...     ([10.5, 11.5], 1.0), ([1.0, 1.0], 0.0), (

In [15]:
kmeans_eval = ClusteringEvaluator()

In [16]:
silhouette = kmeans_eval.evaluate(kmeans_pred)
print(f'Silhouette with squared euclidean distance :{silhouette}')

Silhouette with squared euclidean distance :0.9997530305375207


In [17]:
kmeans = KMeans().setK(3).setSeed(1)
kmeans_model = kmeans.fit(final_data)
kmeans_pred = kmeans_model.transform(final_data)
print('Clustering Results : \n')
kmeans_pred.show()
cluster_centers = kmeans_model.clusterCenters()
print('Cluster Centers : \n{cluster_centers}')
silhouette = kmeans_eval.evaluate(kmeans_pred)
print(f'Silhouette with squared euclidean distance :{silhouette}')

Clustering Results : 

+--------------------+----------+
|            features|prediction|
+--------------------+----------+
|           (3,[],[])|         1|
|(3,[0,1,2],[0.1,0...|         1|
|(3,[0,1,2],[0.2,0...|         2|
|(3,[0,1,2],[9.0,9...|         0|
|(3,[0,1,2],[9.1,9...|         0|
|(3,[0,1,2],[9.2,9...|         0|
+--------------------+----------+

Cluster Centers : 
{cluster_centers}
Silhouette with squared euclidean distance :0.6248737134600261


In [18]:
kmeans = KMeans().setK(4).setSeed(1)
kmeans_model = kmeans.fit(final_data)
kmeans_pred = kmeans_model.transform(final_data)
print('Clustering Results : \n')
kmeans_pred.show()
cluster_centers = kmeans_model.clusterCenters()
print('Cluster Centers : \n{cluster_centers}')
silhouette = kmeans_eval.evaluate(kmeans_pred)
print(f'Silhouette with squared euclidean distance :{silhouette}')

Clustering Results : 

+--------------------+----------+
|            features|prediction|
+--------------------+----------+
|           (3,[],[])|         1|
|(3,[0,1,2],[0.1,0...|         1|
|(3,[0,1,2],[0.2,0...|         2|
|(3,[0,1,2],[9.0,9...|         0|
|(3,[0,1,2],[9.1,9...|         0|
|(3,[0,1,2],[9.2,9...|         3|
+--------------------+----------+

Cluster Centers : 
{cluster_centers}
Silhouette with squared euclidean distance :0.25000000000146066


In [19]:
kmeans = KMeans().setK(5).setSeed(1)
kmeans_model = kmeans.fit(final_data)
kmeans_pred = kmeans_model.transform(final_data)
print('Clustering Results : \n')
kmeans_pred.show()
cluster_centers = kmeans_model.clusterCenters()
print('Cluster Centers : \n{cluster_centers}')
silhouette = kmeans_eval.evaluate(kmeans_pred)
print(f'Silhouette with squared euclidean distance :{silhouette}')

Clustering Results : 

+--------------------+----------+
|            features|prediction|
+--------------------+----------+
|           (3,[],[])|         1|
|(3,[0,1,2],[0.1,0...|         1|
|(3,[0,1,2],[0.2,0...|         2|
|(3,[0,1,2],[9.0,9...|         0|
|(3,[0,1,2],[9.1,9...|         4|
|(3,[0,1,2],[9.2,9...|         3|
+--------------------+----------+

Cluster Centers : 
{cluster_centers}
Silhouette with squared euclidean distance :0.12500000000000008


In [20]:
kmeans = KMeans().setK(6).setSeed(1)
kmeans_model = kmeans.fit(final_data)
kmeans_pred = kmeans_model.transform(final_data)
print('Clustering Results : \n')
kmeans_pred.show()
cluster_centers = kmeans_model.clusterCenters()
print('Cluster Centers : \n{cluster_centers}')
silhouette = kmeans_eval.evaluate(kmeans_pred)
print(f'Silhouette with squared euclidean distance :{silhouette}')

Clustering Results : 

+--------------------+----------+
|            features|prediction|
+--------------------+----------+
|           (3,[],[])|         1|
|(3,[0,1,2],[0.1,0...|         2|
|(3,[0,1,2],[0.2,0...|         3|
|(3,[0,1,2],[9.0,9...|         4|
|(3,[0,1,2],[9.1,9...|         5|
|(3,[0,1,2],[9.2,9...|         0|
+--------------------+----------+

Cluster Centers : 
{cluster_centers}
Silhouette with squared euclidean distance :0.0
